In [ ]:
import sys
from pathlib import Path

path_root = Path( '/project_ghent/HADSP/reservoirpy/')
sys.path.append(str(path_root))
path_root = Path( '/project_ghent/HADSP/hadsp/')
sys.path.append(str(path_root))

In [ ]:
import numpy as np
from scipy import sparse
from tqdm import tqdm

from matplotlib import pyplot as plt
from seaborn import heatmap, color_palette

In [ ]:
from importlib import reload

# Utility functions
from reservoir.utility import show_matrice

# SEED
SEED = 49387

# Reservoir functions

In [ ]:
from reservoir.reservoir import update_reservoir, ridge_regression, train, run
from reservoir.activation_functions import tanh, heaviside, sigmoid

# the activation function choosen for the rest of the experiment
# activation_function = lambda x : sigmoid(2*(x-0.5))
activation_function = lambda x : tanh(x)

plt.plot(np.linspace(0, 2, 100), activation_function(np.linspace(0, 2, 100)))
plt.grid()

# Datasets

Lots of different on availabale : https://towardsdatascience.com/a-data-lakes-worth-of-audio-datasets-b45b88cd4ad

Classification: 
https://arxiv.org/abs/1803.07870

https://github.com/FilippoMB/Time-series-classification-and-clustering-with-Reservoir-Computing

Multivariate:
https://www.timeseriesclassification.com/dataset.php

## Mackey-Glass

In [ ]:
from reservoirpy.datasets import mackey_glass

timesteps = 10000
mg_inputs = mackey_glass(timesteps, tau=17, a=0.2, b=0.1, n=10, x0=1.2, h=1, seed=None)

# Define the time step of your Mackey-Glass system
dt = 0.00001

# Compute the equivalent sampling rate
sampling_rate = 1 / dt

%matplotlib inline
plt.figure(figsize=(16,5))
plt.plot(range(1000), mg_inputs[:1000])
plt.show()

## Japanese voyels

In [ ]:
from reservoirpy.datasets import japanese_vowels

X_train, Y_train, X_test, Y_test = japanese_vowels()

In [ ]:
X_pretrain = np.concatenate(X_train, axis=0)
X_pretrain.shape
filtered_data = X_pretrain.T

## InsectWingbeat

## MELD

https://github.com/declare-lab/MELD


## Free Spoken Digits Dataset

In [ ]:
from datasets.load_datasets import load_FSDD_dataset

data_dir = 'datasets/fsdd/free-spoken-digit-dataset-master/recordings'  # Path to the extracted dataset
sampling_rate, X_train, X_test, Y_train, Y_test = load_FSDD_dataset(data_dir, seed=SEED)
# Check the shapes of the datasets
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)

# New inputs creation with band filters

Spectrograms_vs_Cochleagrams : 
* https://www.researchgate.net/publication/340510607_Speech_recognition_using_very_deep_neural_networks_Spectrograms_vs_Cochleagrams

## Pretrain dataset

In [ ]:
#take a long time (15min with half the samples, instant with 20 which is enought for pretraining)
X_pretrain = np.concatenate(X_train[:20], axis=0)
print(X_pretrain.shape)

## Spectral density and peak selection

In [ ]:
from datasets.multivariate_generation import generate_multivariate_dataset, extract_peak_frequencies

filtered_peak_freqs = extract_peak_frequencies(X_pretrain.flatten(), sampling_rate, nperseg=1024, visualize=False)

## Applying normal band pass filter on data

In [ ]:
X_pretrain_band, X_train_band, X_test_band = generate_multivariate_dataset(filtered_peak_freqs, X_pretrain, X_train, X_test, sampling_rate, nb_jobs=-1, visualize=True)

## Standardizing the amplitudes

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))

# pretrain
filtered_data = scaler.fit_transform(X_pretrain_band)

In [ ]:
# train
X_train_band = [np.array([scaler.fit_transform(time_serie.reshape(-1, 1)).flatten() for time_serie in x]) for x in X_train_band]

In [ ]:
# test
X_test_band = [np.array([scaler.fit_transform(time_serie.reshape(-1, 1)).flatten() for time_serie in x]) for x in X_test_band]

In [ ]:
X_train_band[0].shape

# Generating reservoirs

## Creating from HADSP + bandfilter

### Plot  pretrain dataset

In [ ]:
# Min window size to get all the dynamics ? 
min_window_size = sampling_rate/filtered_peak_freqs[-1]

min_window_size

In [ ]:
#Compute the moving average 
window_size = 5
weights = np.repeat(1.0, window_size)/window_size
ma = np.array([np.convolve(d, weights, 'valid') for d in (filtered_data)])

#CPlot the two for different frequencies
NB_1 = 3
fig, ax = plt.subplots(2, 1, figsize=(24,12))
ax[0].plot(range(500), filtered_data[NB_1, 1000:1500], label='Time serie')
ax[0].plot(range(500), ma[NB_1, 1000:1500], label='Moving average')
NB_2 = 10
ax[0].legend(fontsize=26)
ax[1].plot(range(500), filtered_data[NB_2, 1000:1500], label='Time serie')
ax[1].plot(range(500), ma[NB_2, 1000:1500], label='Moving average')

ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[0].tick_params(axis='both', labelsize=26)
ax[1].tick_params(axis='both', labelsize=26)


# draw vertical lines to represent the window for some points
for x in range(100, 500, 100):
    ax[0].axvspan(x, x+window_size, color='g', alpha=0.2)
for x in range(100, 500, 100):
    ax[1].axvspan(x, x+window_size, color='g', alpha=0.2)

plt.show()

## Construct matrices

### Shared parameters

In [ ]:
import math 

# We want the size of the reservoir to be at least 200
K = math.ceil(200 / filtered_peak_freqs.shape[0])
n = filtered_peak_freqs.shape[0] * K
n

In [ ]:
INCREMENT = 5
VALUE = 0.05
target_rate = 0.7
growth_parameter = 0.3

bias_scaling = 1
input_scaling = 0.1
leaky_rate = 1

### Multivariate matrices

In [ ]:
frequency_bands = np.repeat(filtered_data, K, axis=0)

class TwoDimArrayWrapper:
    def __init__(self, input_data):
        if input_data.ndim != 2:
            raise ValueError("Expected a 2D array.")
        self.input_data = input_data
        self.shape = input_data.shape
        self.size = input_data.shape[1]
        self.flat_data = input_data.flatten()

    def __getitem__(self, key):
        # Handle single element access
        return self.input_data[:, key]
        
frequency_bands = TwoDimArrayWrapper(frequency_bands)
frequency_bands.shape

In [ ]:
import reservoir.reservoir
from reservoir.reservoir import update_reservoir
from reservoir.reservoir import init_matrices
from connexion_generation.utility import compute_synaptic_change
from connexion_generation.bounded_adsp import bounded_adsp

# Initialisation EE
Win_hasp_band, W_hasp_band, bias_hasp_band =  init_matrices(n, 1, 0, seed=SEED)
bias_hasp_band = bias_hasp_band*bias_scaling
Win_hasp_band = Win_hasp_band*input_scaling

def run_HADSP_algorithm(W, Win, bias, input_data, visualize=False):
    # last_state
    state = np.random.uniform(0, 1, n)
    state_history = []
    
    total_add = 0
    total_prun = 0
    add = []
    prun = []
    step=0

    for i in range(INCREMENT*5):
        state = update_reservoir(W, Win, input_data[i], state, leaky_rate, bias, activation_function)
        state_history.append(state)

    # size of simulation 
    number_steps = int((input_data.size-INCREMENT*5)/INCREMENT)
    for k in tqdm(range(number_steps)): 
        delta_z = compute_synaptic_change(state_history[-INCREMENT:], target_rate, growth_parameter, average="WHOLE")
        W, _, nb_new_add, nb_new_prun = bounded_adsp(W, state, delta_z, VALUE)
    
        for i in range(INCREMENT):
            state = update_reservoir(W, Win, input_data[INCREMENT*(k+5)+i], state, leaky_rate, bias, activation_function)
            state_history.append(state)
            
        total_add += nb_new_add
        total_prun += nb_new_prun
        add.append(total_add)
        prun.append(total_prun)
        step += 1
        
    add = np.array(add)
    prun = np.array(prun)

    if visualize:
        plt.figure()
        plt.plot(np.arange(step)*INCREMENT, add, label="total number of added connexion")
        plt.plot(np.arange(step)*INCREMENT, prun, label="total number of prunned connexion")
        plt.plot(np.arange(step)*INCREMENT, add-prun, label="difference")
        plt.plot(np.arange(step)*INCREMENT, [0]*step, linestyle=(0, (1, 10)))
        plt.legend()
        plt.grid()
    return W

W_hasp_band = run_HADSP_algorithm(W_hasp_band, Win_hasp_band, bias_hasp_band, frequency_bands, visualize=False)

We create the matrice with same data and same size but randomly generated

In [ ]:
connectivity_band =  W_hasp_band.count_nonzero() / (W_hasp_band.shape[0] * W_hasp_band.shape[1])
# (hadsp + band)
eigen_adsp = sparse.linalg.eigs(W_hasp_band, k=1, which="LM", maxiter=W_hasp_band.shape[0] * 20, tol=0.1, return_eigenvectors=False)
sr_adsp = max(abs(eigen_adsp))

# 3rd (normal + band)
Win_3, W_3, bias_3 =  init_matrices(n, 1, connectivity_band, sr_adsp)
bias_3= bias_3*bias_scaling
Win_3= Win_3*0.3
eigen_3 = sparse.linalg.eigs(W_3, k=1, which="LM", maxiter=W_3.shape[0] * 20, tol=0.1, return_eigenvectors=False)
sr_3 = max(abs(eigen_3))


### Univariate matrices

In [ ]:
# Initialisation EE
Win_hadsp_uni, W_hadsp_uni, bias_hadsp_uni = init_matrices(n, 1, 0, seed=SEED)
bias_hadsp_uni = bias_hadsp_uni*bias_scaling
Win_hadsp_uni = Win_hadsp_uni*input_scaling

W_hadsp_uni = run_HADSP_algorithm(W_hadsp_uni, Win_hadsp_uni, bias_hadsp_uni, X_pretrain.flatten(), visualize=False)

We create Matrices without HADSP (normal)

In [ ]:
from scipy import stats

# (normal + random)
# We generate a network randomly same connectivity as the HADSP generated one
connectivity_hadsp_uni =  W_hadsp_uni.count_nonzero() / (W_hadsp_uni.shape[0] * W_hadsp_uni.shape[1])
Win_normal, W_normal, bias_normal =  init_matrices(n, 1, connectivity_hadsp_uni)
bias_normal= bias_normal*bias_scaling
Win_normal= Win_normal*0.3

# Set the input matrices for the two HADSPs
#Win_hasp_band = Win_normal
#Win_hadsp_multi = Win_normal

## Spectral radius normalisation

In [ ]:
# (normal + random)
eigen_normal = sparse.linalg.eigs(W_normal, k=1, which="LM", maxiter=W_normal.shape[0] * 20, tol=0.1, return_eigenvectors=False)
sr_normal = max(abs(eigen_normal))

# 2th  (HADSP + random)
eigen_hadsp_uni = sparse.linalg.eigs(W_hadsp_uni, k=1, which="LM", maxiter=W_hadsp_uni.shape[0] * 20, tol=0.1, return_eigenvectors=False)
sr_hadsp_uni = max(abs(eigen_hadsp_uni))

In [ ]:
print(sr_normal)
print(sr_hadsp_uni)
print(sr_3)
print(sr_adsp)

In [ ]:
# Spectral radius normalisation
normal_sr = 1.2
W_normal = W_normal/sr_normal*normal_sr
W_hadsp_uni = W_hadsp_uni/sr_hadsp_uni*normal_sr
W_3 = W_3/sr_3*normal_sr
W_hasp_band = W_hasp_band/sr_adsp*normal_sr

# Performance

## Classification

### Training

In [ ]:
# We want the size of the reservoir to be at least 200
caca = []
caca_test = []
for i in tqdm(range(len(X_train_band))):
    #caca.append(np.repeat(X_train[i], K, axis=1))
    caca.append(np.repeat(X_train_band[i], K, axis=0).T) # axis still depend of X_train shape
for i in tqdm(range(len(X_test_band))):
    #caca_test.append(np.repeat(X_test[i], K, axis=1))
    caca_test.append(np.repeat(X_test_band[i], K, axis=0).T)

In [ ]:
import reservoirpy.nodes
from reservoirpy.nodes import Reservoir, Ridge, Input, ESN
reload(reservoirpy.nodes)
from scipy.sparse import csr_matrix
ridge_coef = 1e-6

source = Input()
# To remember : 
#  For reservoirpy   pre_s = W @ r + Win @ (u + noise_gen(dist=dist, shape=u.shape, gain=g_in)) + bias

In [ ]:
N_JOBS = -1

In [ ]:
from joblib import Parallel, delayed
from sklearn.metrics import mutual_info_score

# Training HADSP + mutlivariate dataset
reservoir_hasp_band = Reservoir(units=n, 
                                W =csr_matrix(W_hasp_band), 
                                Win=csr_matrix(np.diag(Win_hasp_band.toarray().flatten())), 
                                bias=csr_matrix(bias_hasp_band).T, 
                                activation=activation_function,
                                equation='external'
                               )
readout_hasp_band = Ridge(ridge=ridge_coef)
model_hasp_band = ESN(reservoir=reservoir_hasp_band, readout=readout_hasp_band)

states_train_hasp_band = []
def compute_state(x):
    return reservoir_hasp_band.run(x, reset=True)[-1, np.newaxis].flatten()

states_train_hasp_band = Parallel(n_jobs=N_JOBS)(delayed(compute_state)(x) for x in caca)

In [ ]:
# Training random + mutlivariate dataset
reservoir_3 = Reservoir(units=n, 
                        W =csr_matrix(W_3), 
                        Win=csr_matrix(np.diag(Win_3.toarray().flatten())), 
                        bias=csr_matrix(bias_3).T, 
                        activation=activation_function,
                        equation='external'
                       )
readout_3 = Ridge(ridge=ridge_coef)
model_3 = ESN(reservoir=reservoir_3, readout=readout_3)

states_train_3 = []

def compute_state(x):
    return reservoir_3.run(x, reset=True)[-1, np.newaxis].flatten()

states_train_3 = Parallel(n_jobs=N_JOBS)(delayed(compute_state)(x) for x in caca)


### Performance

In [ ]:
#readout_hasp_band.fit(states_train_hasp_band, Y_train)
readout_hasp_band.fit(np.array(states_train_hasp_band), Y_train)

Y_pred_hasp_band = []
def predict(x):
    states = reservoir_hasp_band.run(x, reset=True)
    y = readout_hasp_band.run(states[-1, np.newaxis])
    return y

Y_pred_hasp_band = Parallel(n_jobs=N_JOBS)(delayed(predict)(x) for x in caca_test)

In [ ]:
#readout_3.fit(states_train_3, Y_train)
readout_3.fit(np.array(states_train_3), Y_train)


Y_pred_3 = []
def predict(x):
    states = reservoir_3.run(x, reset=True)
    y = readout_3.run(states[-1, np.newaxis])
    return y

Y_pred_3 = Parallel(n_jobs=N_JOBS)(delayed(predict)(x) for x in caca_test)

In [ ]:
from sklearn.metrics import accuracy_score

Y_pred_class = [np.argmax(y_p) for y_p in Y_pred_hasp_band]
Y_test_class = [np.argmax(y_t) for y_t in Y_test]

score = accuracy_score(Y_test_class, Y_pred_class)

print("Accuracy for HADSP multi: ", f"{score * 100:.3f} %")

In [ ]:
from sklearn.metrics import accuracy_score

Y_pred_class = [np.argmax(y_p) for y_p in Y_pred_3]
Y_test_class = [np.argmax(y_t) for y_t in Y_test]

score = accuracy_score(Y_test_class, Y_pred_class)

print("Accuracy for random multi: ", f"{score * 100:.3f} %")

## Classification for univariate

### Training

In [ ]:
# Create a list to store the arrays

pipi = []
pipi_test = []

for i in tqdm(range(len(X_train))):
    pipi.append(np.repeat(X_train[i], n, axis=1))
    #pipi.append(np.repeat(X_train[i], n, axis=0).T) # axis still depend of X_train shape
    
for i in tqdm(range(len(X_test))):
    pipi_test.append(np.repeat(X_test[i], n, axis=1))
    #pipi_test.append(np.repeat(X_test[i], n, axis=0).T)
    
pipi[0].shape

In [ ]:
# Training HADSP + univariate dataset
reservoir_hadsp_uni = Reservoir(units=n, 
                                W =csr_matrix(W_hadsp_uni), 
                                Win=csr_matrix(np.diag(Win_hadsp_uni.toarray().flatten())), 
                                bias=csr_matrix(bias_hadsp_uni).T, 
                                activation=activation_function,
                                equation='external'
                               )
readout_hadsp_uni = Ridge(ridge=ridge_coef)
model_hadsp_uni = ESN(reservoir=reservoir_hadsp_uni, readout=readout_hadsp_uni)


states_train_hadsp_uni = []

def compute_state(x):
    return reservoir_hadsp_uni.run(x, reset=True)[-1, np.newaxis].flatten()

states_train_hadsp_uni = Parallel(n_jobs=N_JOBS)(delayed(compute_state)(x) for x in pipi)

In [ ]:
# Training HADSP + univariate dataset
reservoir_normal = Reservoir(units=n, 
                                W =csr_matrix(W_normal), 
                                Win=csr_matrix(np.diag(Win_normal.toarray().flatten())), 
                                bias=csr_matrix(bias_normal).T, 
                                activation=activation_function,
                                equation='external'
                               )
readout_normal = Ridge(ridge=ridge_coef)
model_normal = ESN(reservoir=reservoir_normal, readout=readout_normal)


states_train_normal = []

def compute_state(x):
    return reservoir_normal.run(x, reset=True)[-1, np.newaxis].flatten()

states_train_normal = Parallel(n_jobs=N_JOBS)(delayed(compute_state)(x) for x in pipi)

### Performances

In [ ]:
#readout_hasp_uni.fit(states_train_hasp_uni, Y_train)
readout_hadsp_uni.fit(np.array(states_train_hadsp_uni), Y_train)

Y_pred_hadsp_uni = []

def predict(x):
    states = reservoir_hadsp_uni.run(x, reset=True)
    y = readout_hadsp_uni.run(states[-1, np.newaxis])
    return y

Y_pred_hadsp_uni = Parallel(n_jobs=N_JOBS)(delayed(predict)(x) for x in pipi_test)

In [ ]:
#readout_normal.fit(states_train_normal, Y_train)
readout_normal.fit(np.array(states_train_normal), Y_train)

Y_pred_normal = []

def predict(x):
    states = reservoir_normal.run(x, reset=True)
    y = readout_normal.run(states[-1, np.newaxis])
    return y

Y_pred_normal = Parallel(n_jobs=N_JOBS)(delayed(predict)(x) for x in pipi_test)

In [ ]:
from sklearn.metrics import accuracy_score

Y_pred_class = [np.argmax(y_p) for y_p in Y_pred_hadsp_uni]
Y_test_class = [np.argmax(y_t) for y_t in Y_test]

score = accuracy_score(Y_test_class, Y_pred_class)

print("Accuracy for HADSP uni: ", f"{score * 100:.3f} %")

In [ ]:
from sklearn.metrics import accuracy_score

Y_pred_class = [np.argmax(y_p) for y_p in Y_pred_normal]
Y_test_class = [np.argmax(y_t) for y_t in Y_test]

score = accuracy_score(Y_test_class, Y_pred_class)

print("Accuracy for random uni: ", f"{score * 100:.3f} %")

## Prediction ahead

### Training

In [ ]:
import reservoir.reservoir
from reservoir.reservoir import train_ei, train
reload(reservoir.reservoir)

ridge_coef = 1e-7
step_ahead = 30

# To remember
np.zeros(2000)

print(frequency_bands.shape)
print(input_data.shape)
# to generate the evaluation dataset
START_EVAL_STEP = 3000
x_train_bands = frequency_bands[:,:START_EVAL_STEP]
x_train_normal = input_data[:START_EVAL_STEP]
y_train = input_data[step_ahead:START_EVAL_STEP+step_ahead]

# Training random + MG
Wout_normal, b_out_normal, last_state_normal = train(W_normal, Win_normal, bias_normal, x_train_normal, y_train, activation_function, ridge_coef = ridge_coef)

# Training for HADSP + MG
Wout_hadsp_multi, b_out_hadsp_multi, last_state_hadsp_multi = train(W_hadsp_multi, Win_hadsp_multi, bias_hadsp_multi, x_train_normal, y_train, activation_function, ridge_coef = ridge_coef)

# Training random + bandfilter
Wout_3, b_out_3, last_state_3 = train(W_3, Win_3, bias_3, x_train_bands, y_train, activation_function, ridge_coef = ridge_coef)

# Training output HASDP + bandfilter
Wout_hasp_band, b_out_hasp_band, last_state_hasp_band = train(W_hasp_band, Win_hasp_band, bias_hasp_band, x_train_bands, y_train, activation_function, ridge_coef = ridge_coef)

### Prediction

In [ ]:
from reservoir.reservoir import run
reload(reservoir.reservoir)
from reservoir.reservoirpy_util import plot_results, plot_readout


ending_eval_step = 4900
x_eval_bands = frequency_bands[:,START_EVAL_STEP:ending_eval_step]
x_eval_normal = input_data[START_EVAL_STEP:ending_eval_step]
y_target = input_data[START_EVAL_STEP + step_ahead :ending_eval_step + step_ahead]

# Prediction for random + MG
y_pred_normal = run(W_normal, Win_normal, bias_normal, Wout_normal, x_eval_normal, activation_function, b_out_normal, last_state_normal)

# Prediction for HADSP + MG
y_pred_hadsp_multi = run(W_hadsp_multi, Win_hadsp_multi, bias_hadsp_multi, Wout_hadsp_multi, x_eval_normal, activation_function, b_out_hadsp_multi, last_state_hadsp_multi)

# Prediction for random + bandfilter
y_pred_3 = run(W_3, Win_3, bias_3, Wout_3, x_eval_bands, activation_function, b_out_3, last_state_3)

# Prediction for HADSP + bandfilter
y_pred_hasp_band = run(W_hasp_band, Win_hasp_band, bias_hasp_band, Wout_hasp_band, x_eval_bands, activation_function, b_out_hasp_band, last_state_hasp_band)


plot_results(y_pred_3, y_target, sample=400)

In [ ]:
import reservoir.losses
from reservoir.losses import rmse, nrmse
reload(reservoir.losses)

print("nrmse normal        :", float(nrmse(y_target[:300], y_pred_normal[:300])))
print("nrmse hadsp     :", float(nrmse(y_target[:300], y_pred_hadsp_multi[:300])))
print("nrmse random + band :", float(nrmse(y_target[:300], y_pred_3[:300])))
print("nrmse hadsp + band   :", float(nrmse(y_target[:300], y_pred_hasp_band[:300])))

In [ ]:
y_pred_hadsp_multi.shape

In [ ]:
nrmse_array_normal = []
nrmse_array_hadsp_multi = []
nrmse_array_3 = []
nrmse_array_mg = []

for i in range(len(y_target)-100- step_ahead):
    y_target_i = y_target[i:100+i]
    nrmse_array_normal.append(nrmse(y_target_i, y_pred_normal[i:100+i]))
    nrmse_array_hadsp_multi.append(nrmse(y_target_i, y_pred_hadsp_multi[i:100+i]))
    nrmse_array_3.append(nrmse(y_target_i, y_pred_3[i:100+i]))
    nrmse_array_mg.append(nrmse(y_target_i, y_pred_hasp_band[i:100+i]))
    
log10_nrmse_normal = np.log10(nrmse_array_normal)
log10_nrmse_hadsp_multi = np.log10(nrmse_array_hadsp_multi)
log10_nrmse_3 = np.log10(nrmse_array_3)
log10_nrmse_mg = np.log10(nrmse_array_mg)
plt.figure()
plt.plot(log10_nrmse_normal[:1000])
plt.plot(log10_nrmse_hadsp_multi[:1000])
plt.plot(log10_nrmse_3[:1000])
plt.plot(log10_nrmse_mg[:1000])

plt.xlabel('Time steps')
plt.ylabel('Log10 NRMSE')
plt.legend(["HADSP+band", "random", " random + bandfilter", "HADSP"])
plt.show()

# Analysis

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations, permutations
from collections import defaultdict
from scipy.sparse import coo_matrix

def plot_connectivity(coo):
    G = nx.DiGraph()

    # Convert coo_matrix to edge list and add edges to the graph
    for node1, node2 in zip(coo.row, coo.col):
        G.add_edge(node1, node2)

    triad_connections = defaultdict(int)

    # Get all sets of 3 nodes
    for nodes in combinations(G.nodes, 3):
        # For each node in the triad, count its incoming and outgoing connections within the triad
        connectivity_states = []
        for node in nodes:
            in_edges = sum([G.has_edge(other, node) for other in nodes if other != node])
            out_edges = sum([G.has_edge(node, other) for other in nodes if other != node])
            connectivity_states.append((in_edges, out_edges))
        # Sort the connectivity states so that equivalent triads have the same key, regardless of node order
        connectivity_states = tuple(sorted(connectivity_states))
        triad_connections[connectivity_states] += 1

    # Sort results by total number of connections
    sorted_results = sorted(((sum(in_edges for in_edges, out_edges in key) + sum(out_edges for in_edges, out_edges in key), key) for key in triad_connections.items()), key=lambda x: x[0])
    sorted_labels = [str(key) for total, key in sorted_results]
    sorted_values = [triad_connections[key] for total, key in sorted_results]

    # Plot results
    plt.bar(range(len(sorted_labels)), sorted_values, tick_label=sorted_labels)
    plt.xlabel('Triad connectivity state')
    plt.ylabel('Count')
    plt.title('Triad Connection Count in Graph')
    plt.xticks(rotation=90)
    plt.show()


plot_connectivity(W_normal)

In [ ]:
plot_connectivity(W_hasp_band)

In [ ]:
time_series = filtered_data
# Create a matrix to hold the mutual information between each pair of time series
mi_matrix = np.zeros((len(time_series), len(time_series)))

    
# Function to compute mutual information
def compute_mi(i, j):
    # Discretize the data
    time_series_i_discrete = np.floor(time_series[i] * 10).astype(int)
    time_series_j_discrete = np.floor(time_series[j] * 10).astype(int)
    
    # Compute mutual information
    mi = mutual_info_score(time_series_i_discrete, time_series_j_discrete)
    
    return i, j, mi

# Create a list of all pairs of indices
indices = [(i, j) for i in range(len(time_series)) for j in range(i, len(time_series))]



# Use joblib to parallelize the computation
results = Parallel(n_jobs=-1, backend="threading", verbose=1)(delayed(compute_mi)(i, j) for i, j in indices)

# Store the results in the matrix
for i, j, mi in results:
    mi_matrix[i, j] = mi
    mi_matrix[j, i] = mi

pbar.close()

heatmap(mi_matrix)